In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras.layers as L
from keras import regularizers, optimizers
from collections import Counter
import keras
from keras import Model
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")
from keras.models import load_model

In [ ]:
# Load the labelling csv file 
label = pd.read_csv("../input/landmark-recognition-2020/train.csv")
label.head()

In [ ]:
print("The total number of pictures in the dataset:", len(label))
print("The total number of landmarks in the dataset:", label.landmark_id.nunique())

In [ ]:
counts = label['landmark_id'].value_counts().sort_values(ascending=False)
counts[0:5]

In [ ]:
# Lets see the classes with least amount of images:
below = counts[counts < 10].index.shape[0]
below

In [ ]:
# We have 41637 classes with less than 10 images. Inclusding these images in the model may create noise
# as the number of these images are very less. therefore removing these images from our original df

selected_classes = counts[counts >= 20].index
label = label.loc[label.landmark_id.isin(selected_classes)]
print(label.shape)

In [ ]:
# Approach 1:
# For this project, lets limit our scope to only 1000 images each of 5 top occuring classes.
# from these 5000 images, we will split training, validation in the ratio 75:25

In [ ]:
label['landmark_id'] = label.landmark_id.astype(int)

In [ ]:
#selected_df = label.loc[label['landmark_id'].isin([int(x) for x in list(counts[0:5].index)])]

# Now subset only 1000 images from each of the 5 categories:
#selected_df = selected_df.groupby('landmark_id').apply(lambda x: x.sample(n=1000, random_state =13)).reset_index(drop = True)
#Counter(selected_df['landmark_id'])

In [ ]:
# Approach 2:
# Lets select specified percentage of images from each class after removing the noise:
selected_df = label.groupby("landmark_id", group_keys=False).apply(lambda x: x.sample(frac = 0.3, random_state = 123))
len(selected_df)

In [ ]:
# calculate the number of classes from the sratified dataset above
count = Counter(selected_df.landmark_id.values)
num_classes = len(count)
num_classes

In [ ]:
# Image generator: Lets load these images using image data generator and perform train test split

In [ ]:
selected_df['landmark_id'] = selected_df.landmark_id.astype(str)
selected_df['id'] = selected_df.id.str[0] + '/' + selected_df.id.str[1] + '/' + selected_df.id.str[2]+'/' + selected_df.id + '.jpg'

In [ ]:
val_split = 0.25
batch_size = 32
img_width = img_height = 192

In [ ]:
#  Use flow_from_dataframe to generate train data set, test, validation data set

datagen=ImageDataGenerator(validation_split=val_split,rescale=1. / 255)

train_generator=datagen.flow_from_dataframe(dataframe=selected_df,
                                            directory="/kaggle/input/landmark-recognition-2020/train/",
                                            x_col="id",
                                            y_col="landmark_id",
                                            subset="training",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(img_height, img_width))

valid_generator=datagen.flow_from_dataframe(
dataframe=selected_df,
directory="/kaggle/input/landmark-recognition-2020/train/",
x_col="id",
y_col="landmark_id",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(img_height, img_width))

In [ ]:
def my_model(input_shape, num_classes, dropout, learning_rate = 0.0002):

    base_model = Xception(input_shape=input_shape,weights='imagenet', include_top=False)
    #base_model.load_weights("https://storage.googleapis.com/tensorflow/keras-applications/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5")


    x = base_model.output
    x = L.Dropout(dropout)(x)
    x = L.SeparableConv2D(256, kernel_size=(3, 3), activation='relu',kernel_initializer = tf.keras.initializers.he_uniform(seed=1))(x)
    x = L.BatchNormalization()(x)
    x = L.SeparableConv2D(128, kernel_size=(3, 3), activation='relu',kernel_initializer = tf.keras.initializers.he_uniform(seed=3))(x)
    x = L.BatchNormalization()(x)
    x = L.SeparableConv2D(num_classes,kernel_size = (1,1), depth_multiplier=1, activation = 'relu',
                kernel_initializer = tf.keras.initializers.he_uniform(seed=0),
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.1, l2=0.01)
                )(x)
    x = L.GlobalMaxPooling2D()(x)
    x = L.BatchNormalization()(x)
    x = L.Flatten()(x)

    pred = L.Dense(num_classes, activation = 'softmax')(x)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs = base_model.input,outputs = pred,name='model')

    model.compile(loss='categorical_crossentropy',experimental_steps_per_execution=8, optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01), metrics='categorical_accuracy')

    model.summary()
    return model

In [ ]:
model = my_model(input_shape = (img_width, img_height, 3), num_classes = num_classes, dropout = 0.5)

In [ ]:
epochs = 30 # Defining epochs for the model
train_samples  = int(len(selected_df)*(1-val_split))//batch_size
validation_samples  = int(len(selected_df)*val_split)//batch_size

print(train_samples)
print(validation_samples)

In [ ]:
# Define call backs and metrics:

checkpointer = ModelCheckpoint('basic_cnn.h5', monitor='val_loss', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10)

METRICS = [
    keras.metrics.Accuracy(name= "accuracy"),
    keras.metrics.Precision(name = "precision"),
    keras.metrics.Recall(name = 'recall'),
    keras.metrics.AUC(name = 'auc'),
]

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        callbacks=[checkpointer, early_stopping],
        use_multiprocessing=True,
        verbose=1,
        validation_data=valid_generator,
        validation_steps=validation_samples // batch_size,)

model.save("basic_cnn.h5")

# Fine Tuning the Xception model

In [ ]:
for layer in model.layers:
    layer.trainable = True
    
model.compile(loss='categorical_crossentropy', experimental_steps_per_execution=8, optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), metrics='categorical_accuracy')
model.summary()

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        callbacks=[checkpointer, early_stopping],
        use_multiprocessing=True,
        verbose=1,
        validation_data=valid_generator,
        validation_steps=validation_samples // batch_size)

# Evaluating the model

In [ ]:
scores = model.evaluate_generator(valid_generator, validation_samples, use_multiprocessing=True, verbose=1)
scores
#print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

In [ ]:
submission = pd.read_csv("/kaggle/input/landmark-recognition-2020/sample_submission.csv")
submission["id"] = submission.id.str[0]+"/"+submission.id.str[1]+"/"+submission.id.str[2]+"/"+submission.id+".jpg"
best_model = load_model("basic_cnn.h5")

test_gen = ImageDataGenerator().flow_from_dataframe(
    submission,
    directory="/kaggle/input/landmark-recognition-2020/test/",
    x_col="id",
    y_col=None,
    weight_col=None,
    target_size=(img_width, img_height),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    batch_size=1,
    shuffle=True,
    subset=None,
    interpolation="nearest",
    validate_filenames=False)

In [ ]:
y_pred_one_hot = best_model.predict_generator(test_gen, verbose=1, steps=len(submission))

In [ ]:
y_pred = np.argmax(y_pred_one_hot, axis=-1)
y_prob = np.max(y_pred_one_hot, axis=-1)
print(y_pred.shape, y_prob.shape)

In [ ]:
y_uniq = np.unique(selected_df.landmark_id.values)
y_pred = [y_uniq[Y] for Y in y_pred]

In [ ]:
# Extracting best and worst classficiations from predictions
temp_sub = submission

for i in range(len(temp_sub)):
    temp_sub.loc[i, "landmarks"] = str(y_pred[i])

temp_sub.insert(2, "pred", y_prob)    

worst_preds = temp_sub.sort_values(by=['pred'])
best_preds = temp_sub.sort_values(by=['pred'], ascending=False)

In [ ]:
worst_preds[0:5]

In [ ]:
len(best_preds[best_preds['pred'] > 0.8])